In [ ]:
from JavaClones.Vec2 import Vec2
from Plotting.Plotter import PlotType, Plotter

In [ ]:
v_max = 2.0
a_max = 3.0

In [ ]:
Plotter.plot(
    s0=0.2,
    s1=1.5,
    v0=0.3,
    v_max=v_max,
    a_max=a_max,
    tt=None,
    primary_direction=None,
    plot_type=PlotType.TRAJ,
    save_fig=False,
    show_fig=True
)

In [ ]:
Plotter.plot(
    s0=Vec2.zero(),
    s1=Vec2(1.0, 1.0),
    v0=Vec2(v_max, 0.0),
    v_max=v_max,
    a_max=a_max,
    tt=1.2,
    primary_direction=None,
    plot_type=PlotType.TRAJ,
    save_fig=False,
    show_fig=True
)

In [ ]:
distances = [Vec2(1.5, 3)]
initial_velocities = [Vec2(0, 1)]
target_times = [1.9, None]

for d in sorted(set(distances)):
    for i_v in sorted(set(initial_velocities)):
        for tt in set(target_times):
            Plotter.plot(
                s0=Vec2.zero(),
                s1=d,
                v0=i_v,
                v_max=v_max,
                a_max=a_max,
                tt=tt,
                primary_direction=None,
                plot_type=PlotType.TRAJ,
                save_fig=True,
                show_fig=True
            )

In [ ]:
test_values = [
    (0.0, 0.0, 0.0),
    (0.0, 1.0, 0.0),
    (0.0, 1.0, 1.0),
    (0.0, 1.0, 1.1),
    (0.5, 2.0, 0.5),
    (0.5, 2.0, 0.92),
    (1.0, 1.0, 0.5),
    (1.0, 1.0, 0.6),
    (1.0, 1.0, 1.0),
    (2.0, 2.0, 1.1),
    (2.0, 2.0, 2.0),
    (1.0, 0.0, 1.0),
]

for values in test_values:
    Plotter.plot(
        s0=0,
        s1=values[0],
        v0=values[1],
        v_max=v_max,
        a_max=a_max,
        tt=values[2],
        primary_direction=None,
        plot_type=PlotType.TRAJ,
        save_fig=False,
        show_fig=True
    )

In [ ]:
test_values = [
    (0.5, 1.5, 0.1, "trap_too_slow_2.png"),
    (0.2, 1.5, 0.1, "line_too_slow.png"),
    (0.2, 0, 0.4, "tria_too_slow.png"),
    (0.2, 0, 0.6, "tria_direct.png"),
    (0.2, 0, 0.8, "tria_early.png"),
    (1, 0, 1, "trap_too_slow_1.png"),
    (1, 0, 1.5, "trap_direct.png"),
    (1, 0, 2, "trap_early.png"),
    (0.2, 2, 2, "overshoot.png"),
]

for values in test_values:
    Plotter.plot(
        s0=0,
        s1=values[0],
        v0=values[1],
        v_max=v_max,
        a_max=a_max,
        tt=values[2],
        primary_direction=None,
        plot_type=PlotType.TRAJ_NO_ACC,
        save_fig=values[3],
        show_fig=True,
        v_max_1d=1,
        a_max_1d=1.5,
    )

In [ ]:
import math

s = 1.028468852217555351558075926732271909713745117187500000000000
v0 = 1.032920985147269377080192498397082090377807617187500000000000
factor = 0.1467304745
factor2 = factor + 0.0604
tt = 0.903802978547136981468668182060355320572853088378906250000000
print("Total times:")
print("{:.3f}".format(Plotter.plot(
    s0=0,
    s1=s,
    v0=v0,
    v_max=v_max,
    a_max=a_max,
    tt=tt,
    primary_direction=None,
    plot_type=PlotType.TRAJ_NO_ACC,
    v_max_1d=v_max * factor,
    a_max_1d=a_max * factor,
    save_fig="none_convexity_1.png"
).get_total_time()))
print("{:.3f}".format(Plotter.plot(
    s0=0,
    s1=s,
    v0=v0,
    v_max=v_max,
    a_max=a_max,
    tt=tt,
    primary_direction=None,
    plot_type=PlotType.TRAJ_NO_ACC,
    v_max_1d=v_max * factor2,
    a_max_1d=a_max * factor2,
    save_fig="none_convexity_2.png"
).get_total_time()))
print()

print("Angles [deg]:")
print(f"{math.acos(factor) * 180 / math.pi:.1f}")
print(f"{math.acos(factor2) * 180 / math.pi:.1f}")
print()

print("Max Vel and Max Acc:")
print(f"{v_max * factor:.3f}, {a_max * factor:.3f}")
print(f"{v_max * factor2:.3f}, {a_max * factor2:.3f}")

In [ ]:
test_values = [
    ((-1.5, -3), (2, 0), 2.3),
    ((1, 0.5), (0, 0), 1),
    ((-1.5, -3), (0, 0), 2.2),
    ((-1, 0.5), (0, 0), 1),
    ((-1.5, -3), (0, 0), 2.6),
    ((-1.5, -3), (0, 0), 2.5),
    ((-1.5, -3), (0, 0), 2.4),
    ((-1.5, -3), (0, 0), 2.3),
]

for values in test_values:
    Plotter.plot(
        s0=0,
        s1=Vec2(*values[0]),
        v0=Vec2(*values[1]),
        v_max=v_max,
        a_max=a_max,
        tt=values[2],
        primary_direction=None,
        plot_type=PlotType.TRAJ,
        save_fig=False,
        show_fig=True
    )